<a href="https://colab.research.google.com/github/lpscby/MachineLearning101/blob/main/BY_ML101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第一课: Tech with Tim - Linear Regression
课程内容:使用UCL的[Student Performance数据集](https://archive.ics.uci.edu/ml/datasets/Student+Performance),使用线性回归模型进行拟合和分数预测,并检查预测效果.

In [ ]:
#上传本地文档
from google.colab() import files

uploaded = files.upload()

In [ ]:
#读取CSV文档信息,并做检查,熟悉data操作方法
import pandas as pd

data = pd.read_csv('/content/student-mat-comma.csv')

print('Watch the data type: ', type(data))
print('Watch the data head: ', '\n', data.head())

In [ ]:
#对data进行切分,便于下一步输入模型进行拟合
import numpy as np

data = data[["G1", "G2", "G3", "studytime", "failures", "absences"]]